## Library Loadings

In [1]:
import boto
from boto.manage.cmdshell import sshclient_from_instance
import time
import pandas as pd
import os

## S3 Bucket Connections

In [2]:
access = pd.read_csv('rootkey.csv', sep = '=', header = None)
s3 = boto.connect_s3(aws_access_key_id=access.loc[0,1], 
                     aws_secret_access_key=access.loc[1,1])

## EC2 Connection and script launch

##### Connection to ec2

In [3]:
ec2 = boto.connect_ec2(aws_access_key_id=access.loc[0,1], 
                       aws_secret_access_key=access.loc[1,1])

In [4]:
reservations = ec2.get_all_instances()
instance = reservations[0].instances[0]

In [5]:
# ##To do Only Once : 
#key_pair = ec2.create_key_pair('ec2-romain5-key')
#key_pair.save('~/.ssh')
#group = ec2.create_security_group('root', 'A group that allows SSH access')
#group.authorize('tcp', 22, 22, '0.0.0.0/0') # to properly authorize ssh

##### Run Instance

In [19]:
## Running a classical Amazon t2.micro Linux Instance
reservation = ec2.run_instances(image_id='ami-60b6c60a', 
                                key_name='ec2-romain5-key', 
                                #key_name = 'ec2-romain2-key-oldMac',
                                instance_type = 'c3.8xlarge',
                                #instance_type = 't2.micro',
                                security_groups = ['root'])

instance = reservation.instances[0]
print('waiting for instance')
while instance.state != 'running':
    #print ('.', end = "")
    print('.'),
    time.sleep(5)
    instance.update()
print('done')

waiting for instance
. . . done


##### Upload Script

In [20]:
import boto.manage.cmdshell
 
def upload_file(instance, key, username, local_filepath, remote_filepath):
    """
    Upload a file to a remote directory using SFTP. All parameters except
    for "instance" are strings. The instance parameter should be a
    boto.ec2.instance.Instance object.
 
    instance        An EC2 instance to upload the files to.
    key             The file path for a valid SSH key which can be used to
                    log in to the EC2 machine.
    username        The username to log in as.
    local_filepath  The path to the file to upload.
    remote_filepath The path where the file should be uploaded to.
    """
    ssh_client = boto.manage.cmdshell.sshclient_from_instance(
        instance,
        key,
        user_name=username
    )
    ssh_client.put_file(local_filepath, remote_filepath)

##### Install.sh Transfer

In [21]:
# upload_file(instance, 
#             '/Users/romainbui/.ssh/ec2-romain5-key.pem', 
#             'ec2-user', 
#             '/Users/romainbui/Documents/Harvard/CS-181-Machine_Learning/Practicals_GIT/cs181-practicals/practical3/install.sh', 'install.sh')

In [23]:
upload_file(instance, 
            '/Users/romainbui/.ssh/ec2-romain5-key.pem', 
            'ec2-user', 
            '/Users/romainbui/Documents/Harvard/CS-181-Machine_Learning/Practicals_GIT/cs181-practicals/practical3/install2.sh', 'install2.sh')

##### main.py Transfer

In [24]:
# upload_file(instance, 
#             '/Users/romainbui/.ssh/ec2-romain2-key.pem', 
#             'ec2-user', 
#             '/Users/romainbui/Documents/Harvard/CS-181-Machine_Learning/Practicals_GIT/cs181-practicals/practical3/main.py', 'main.py')

##### main2.py Transfer

In [38]:
upload_file(instance, 
            '/Users/romainbui/.ssh/ec2-romain5-key.pem', 
            'ec2-user', 
            '/Users/romainbui/Documents/Harvard/CS-181-Machine_Learning/Practicals_GIT/cs181-practicals/practical3/main2.py', 'main2.py')

##### Transfer .npy

In [26]:
upload_file(instance, 
            '/Users/romainbui/.ssh/ec2-romain5-key.pem', 
            'ec2-user', 
            '/Users/romainbui/Documents/Harvard/CS-181-Machine_Learning/Practicals_GIT/cs181-practicals/practical3/Data/EC2/support.npy', 'support.npy')

In [27]:
upload_file(instance, 
            '/Users/romainbui/.ssh/ec2-romain5-key.pem', 
            'ec2-user', 
            '/Users/romainbui/Documents/Harvard/CS-181-Machine_Learning/Practicals_GIT/cs181-practicals/practical3/Data/EC2/Nsupport.npy', 'Nsupport.npy')

##### Transfer user_avg

In [30]:
upload_file(instance, 
            '/Users/romainbui/.ssh/ec2-romain5-key.pem', 
            'ec2-user', 
            '/Users/romainbui/Documents/Harvard/CS-181-Machine_Learning/Practicals_GIT/cs181-practicals/practical3/Data/EC2/user_avg.csv', 'user_avg.csv')

##### Transfer Root Key

In [28]:
upload_file(instance, 
            '/Users/romainbui/.ssh/ec2-romain5-key.pem', 
            'ec2-user', 
            '/Users/romainbui/Documents/Harvard/CS-181-Machine_Learning/Practicals_GIT/cs181-practicals/practical3/rootkey.csv', 'rootkey.csv')

## End EC2 : DO NOT FORGET

In [18]:
instance.terminate()